In [23]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector

In [24]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
detector = FaceMeshDetector(maxFaces=1)

In [25]:
while True:
    ret, img = cap.read()
    img, faces = detector.findFaceMesh(img, draw=False)
    
    if faces:
        face = faces[0]
        
        pointLeft = face[145]
        pointRight = face[374]
        
        cv2.line(img, pointLeft, pointRight, (0, 200, 0), 3)
        
        cv2.circle(img, pointLeft, 5, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, pointRight, 5, (255, 0, 255), cv2.FILLED)
        
        w, _ = detector.findDistance(pointLeft, pointRight)
        
        
        # Finding the Focal Length
        W = 6.3
        d = 50       
        f = (w*d)/W
        
        print(f)
        
        # Finding distance
        d = (W*f)/w
        
        print(d)
        
        cvzone.putTextRect(img, f'Depth: {d}cm', (face[10][0]-75, face[10][1]-50),
                           scale = 2)                
        
                
    cv2.imshow('Image', img)
    cv2.waitKey(1)

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [18]:
import cv2
import time

def test_camera(index):
    cap = cv2.VideoCapture(index, cv2.CAP_DSHOW)  # Usando CAP_DSHOW para compatibilidad
    if not cap.isOpened():
        print(f"No se pudo abrir la cámara en el índice {index}")
        return

    print(f"Mostrando la cámara en el índice {index}...")
    success, img = cap.read()
    if success:
        cv2.imshow(f'Cámara Índice {index}', img)
        cv2.waitKey(3000)  # Muestra la imagen por 3000 milisegundos (3 segundos)
    else:
        print(f"No se pudo capturar imagen en la cámara índice {index}")

    cap.release()
    cv2.destroyAllWindows()

# Prueba cada cámara disponible
for index in [0, 1]:
    test_camera(index)
    time.sleep(1)  # Espera un segundo antes de probar la próxima cámara para evitar conflictos


Mostrando la cámara en el índice 0...
No se pudo abrir la cámara en el índice 1
